# Lab 1B: Project Spoke Deployment

Deploy a **Project Team (Spoke)** that connects to the central Landing Zone via APIM gateway.

## What Gets Deployed

| Resource | Purpose |
|----------|--------|
| AI Foundry Account | Project team's AI account |
| Foundry Project | Workspace for the team |
| APIM Connection | Gateway connection to landing zone models (API Key auth) |

## Key Requirements

- **APIM Connection Auth**: Uses API Key authentication (AAD not yet supported for APIM connections)
- **Connection Target**: Must include the API path (e.g., `https://apim.azure-api.net/openai`)
- **Static Model Discovery**: Models defined in connection metadata (no dynamic discovery endpoint needed)

> ⚠️ **Prerequisite**: Complete **Lab 1A** first to deploy the Landing Zone

## Step 1: Load Landing Zone Configuration

In [ ]:
import os

# Load the .env file from the workspace root (no external dependency)
env_file = '/workspaces/getting-started-with-foundry/.env'
with open(env_file) as f:
    for line in f:
        line = line.strip()
        if line and not line.startswith('#') and '=' in line:
            key, value = line.split('=', 1)
            os.environ[key] = value

AI_ENDPOINT = os.environ['AI_ENDPOINT']
APIM_URL = os.environ['APIM_URL']
APIM_KEY = os.environ['APIM_KEY']
MODEL_NAME = os.environ['MODEL_NAME']

print(f"Landing Zone AI Endpoint: {AI_ENDPOINT}")
print(f"Landing Zone APIM URL:    {APIM_URL}")
print(f"APIM Key:                 {APIM_KEY[:8]}... (hidden)")
print(f"Model Name:               {MODEL_NAME}")

## Step 2: Set Spoke Variables

In [2]:
SPOKE_RG = "foundry-child-1"
LOCATION = "eastus2"
RG = "foundry-lz-parent"

## Step 3: Create Resource Group

In [ ]:
!az group create -n "{SPOKE_RG}" -l "{LOCATION}" -o table

## Step 4: Deploy Spoke Infrastructure

⏱️ Takes ~2-3 minutes

In [ ]:
import subprocess
PRINCIPAL_ID = subprocess.run('az ad signed-in-user show --query id -o tsv', shell=True, capture_output=True, text=True).stdout.strip()
!az deployment group create -g "{SPOKE_RG}" --template-file main.bicep \
    -p deployerPrincipalId="{PRINCIPAL_ID}" \
    -p apimUrl="{APIM_URL}" \
    -p modelName="{MODEL_NAME}" \
    -p apimSubscriptionKey="{APIM_KEY}" \
    -o table

## Step 5: Get Spoke Outputs

In [ ]:
import subprocess, json
from pathlib import Path

r = subprocess.run(f'az deployment group show -g "{SPOKE_RG}" -n main --query properties.outputs -o json', shell=True, capture_output=True, text=True)
out = json.loads(r.stdout)

SPOKE_ACCOUNT = out['accountName']['value']
SPOKE_ENDPOINT = out['accountEndpoint']['value']
SPOKE_PROJECT = out['projectName']['value']
APIM_CONNECTION = out['apimConnectionName']['value']

print(f"Spoke Account:      {SPOKE_ACCOUNT}")
print(f"Spoke Endpoint:     {SPOKE_ENDPOINT}")
print(f"Project Name:       {SPOKE_PROJECT}")
print(f"APIM Connection:    {APIM_CONNECTION}")

# Append spoke outputs to .env file
env_file = Path("/workspaces/getting-started-with-foundry/.env")
with open(env_file, 'a') as f:
    f.write(f"\n# Spoke Project Outputs\n")
    f.write(f"SPOKE_ACCOUNT={SPOKE_ACCOUNT}\n")
    f.write(f"SPOKE_ENDPOINT={SPOKE_ENDPOINT}\n")
    f.write(f"SPOKE_PROJECT={SPOKE_PROJECT}\n")
    f.write(f"APIM_CONNECTION={APIM_CONNECTION}\n")
print(f"\n✅ Outputs appended to {env_file}")

## Step 6: Connect to Project and Test Agent

Now test the spoke by creating an agent that uses the APIM gateway connection.

In [6]:
!pip install azure-ai-projects==2.0.0b2 azure-identity agent-framework-azure-ai==1.0.0b251223 -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

# Create the AI Project client using the spoke project endpoint
# The project endpoint format is: https://<account>.services.ai.azure.com/api/projects/<project-name>
credential = DefaultAzureCredential()

# Construct the project endpoint from account endpoint
# Account endpoint: https://<account>.cognitiveservices.azure.com/
# Project endpoint: https://<account>.services.ai.azure.com/api/projects/<project-name>
account_host = SPOKE_ENDPOINT.replace("https://", "").replace(".cognitiveservices.azure.com/", "")
PROJECT_ENDPOINT = f"https://{account_host}.services.ai.azure.com/api/projects/{SPOKE_PROJECT}"

project_client = AIProjectClient(
    credential=credential,
    endpoint=PROJECT_ENDPOINT
)

# Get the OpenAI-compatible client for responses API
openai_client = project_client.get_openai_client()

print(f"✅ Connected to Spoke Project: {SPOKE_PROJECT}")
print(f"✅ Project Endpoint: {PROJECT_ENDPOINT}")
print(f"✅ OpenAI client ready for responses API")

## Step 7: Create and Test Hello World Agent

Create a simple agent that uses the APIM gateway connection to the landing zone models.

In [8]:
from azure.ai.projects.models import PromptAgentDefinition

# The gateway model format is: <connection-name>/<model-id>
GATEWAY_MODEL = f"{APIM_CONNECTION}/{MODEL_NAME}"
HELLO_AGENT_NAME = "hello-world-agent"

print(f"Creating Hello World agent...")
print(f"Using gateway model: {GATEWAY_MODEL}")

# Create a versioned agent using the APIM gateway connection
hello_agent = project_client.agents.create_version(
    agent_name=HELLO_AGENT_NAME,
    definition=PromptAgentDefinition(
        model=GATEWAY_MODEL,
        instructions="You are a friendly assistant. When someone greets you, respond with a cheerful greeting that includes the phrase 'Hello from the Spoke via APIM gateway!' somewhere in your response. Keep responses brief."
    ),
)

print(f"✅ Agent created: {hello_agent.name} v{hello_agent.version}")
print(f"   Model: {GATEWAY_MODEL}")

Creating Hello World agent...
Using gateway model: landing-zone-apim/gpt-4.1-mini
✅ Agent created: hello-world-agent v1
   Model: landing-zone-apim/gpt-4.1-mini


In [9]:
# Invoke the agent using the Responses API
query = "Hello! I'm testing the BYO Gateway feature from the spoke project. Can you confirm this is working?"
print(f"User: {query}")

# Use the Responses API with agent_reference (must include version)
response = openai_client.responses.create(
    input=query,
    extra_body={
        "agent": {
            "name": hello_agent.name,
            "version": hello_agent.version,
            "type": "agent_reference"
        }
    }
)

# Extract the response text
agent_response = response.output_text if hasattr(response, 'output_text') else str(response.output)

print("\n" + "="*50)
print("CONVERSATION:")
print("="*50)
print(f"\n🧑 User:\n  {query}")
print(f"\n🤖 Agent:\n  {agent_response}")

User: Hello! I'm testing the BYO Gateway feature from the spoke project. Can you confirm this is working?

CONVERSATION:

🧑 User:
  Hello! I'm testing the BYO Gateway feature from the spoke project. Can you confirm this is working?

🤖 Agent:
  Hello from the Spoke via APIM gateway! Your message came through perfectly, so it looks like the BYO Gateway feature is working fine. How can I assist you further?


In [21]:
# !az group delete -n "{SPOKE_RG}" --yes --no-wait

## Done!

Your project spoke is now connected to the landing zone via the BYO APIM Gateway. The team can:
- Access shared models via APIM gateway connection
- Build agents using `<connection-name>/<model-id>` format
- Benefit from centralized rate limiting and policies
- Have their usage tracked for cost allocation

### Key Concepts

| Concept | Description |
|---------|-------------|
| Gateway Model Format | `<connection-name>/<model-id>` (e.g., `landing-zone-apim/gpt-4.1-mini`) |
| Connection Auth | API Key (stored securely in connection credentials) |
| Static Model Discovery | Models defined in connection metadata via `models` JSON string |
| PromptAgentDefinition | Defines agent with model, instructions, and tools |
| Responses API | Invoke agents via `openai_client.responses.create()` with `agent_reference` |

## Cleanup (Optional)

In [22]:
# Clean up the Hello World agent
# project_client.agents.delete(agent_name=HELLO_AGENT_NAME)
# print(f"✅ Hello World agent '{HELLO_AGENT_NAME}' deleted")
# !az group delete -n "{SPOKE_RG}" --yes --no-wait